# Identify Factors that Predict Intro CS Experience Based on Gender


## Problem Statement

With this project, the problem I am interested in investigating is the gendered experience of the two CS classes in the study. Using machine learning algorithms, I want to identify the leading indicators of the experience broken down by gender in introductory CS at an elite research university like Berkeley.

To solve this problem, I will undertake the following course of action:
1. Explore the dataset.
    - Usually, I would explore the dataset to ensure its integrity and understand the context. But in this case, I will skip this step since I designed the study and collected the data, as such, I am well versed of the context. Further, I have done previous work on this dataset, so I know its boundaries.
2. Identify features that may be used. 
    - If possible, engineer features that might provide greater discrimination.
3. With the understanding that this a `classification` task, explore a couple of classifiers that might be well suited for the problem at hand.
4. Once a classifier has been selected, tune it for optimality.

In [ ]:
%pylab inline

In [ ]:
# Import libraries
from __future__ import division
import sys
sys.path.append('tools/')

import numpy as np
import pandas as pd
import pickle
import tools

   
# Graphing Libraries
import matplotlib.pyplot as pyplt
import seaborn as sns
sns.set_style("white")    

### Explore the dataset

Let's go ahead and read in the student dataset. There are two functions that support this dataset:
- `dataLookUp(surveyItemCode)` This function take a string that is coded survey item. For example if you execute `dataLookUp(atcs_1)`, it prints out the corresponding survey question, I like to use computer science to solve problems. 
- `dataDescr()` This function gives you a general introduction to the dataset.


In [ ]:
dataset = tools.preprocess()

In [ ]:
tools.dataDescr()

In [ ]:
dataset = dataset.query('gender == "Female" or gender == "Male"') #load rows with binary gender
dataset = dataset.reset_index(drop=True)


print '{:d}{:20}{:d}{:20}'.format(len(dataset.query('gender == "Male"')), ' male students, and ',
                                  len(dataset.query('gender == "Female"')),
                                  ' female students consented to participate in this study')
                        

## Preprocess Data

### Preprocess feature columns
To prepare the data for classification, I need to devise a scheme to transform all features into numeric data. This dataset as several non-numeric columns that need converting. Many of them are simply `yes`/`no`, e.g. `prcs_2`. I can reasonably convert these into `1`/`0` (binary) values. For the columns whose values are `Nan`, I will convert these to `0`. 

In [ ]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['Yes', 'No'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            # e.g. 'reason' => 'reason_class_Interested' , 'reason_class_Other'
            col_data = pd.get_dummies(col_data, prefix=col)

        outX = outX.join(col_data)  # collect column(s) in output dataframe
        outX.fillna(0, inplace=True) # make sure all NaN <missing> values are set to 0

    return outX



In [ ]:
dataset = preprocess_features(dataset)
print "Processed feature columns ({}):- {}".format(len(dataset.columns), list(dataset.columns))


### Preprocess feature columns - Rename columns

There are some columns that have spaces in their names, these makes it difficult for the tree plotting algorithms that we will be using later to graph these features. As a result, we will change these spaces to hypens.

In [ ]:
dataset.rename(columns = {'grade_B or above':'grade_B_or_above'}, inplace = True)
dataset.rename(columns = {'grade_B or below':'grade_B_or_below'}, inplace = True)

### Scaling
Linearly scale each attribute to the range [0, 1].



In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_scaled = pd.DataFrame(scaler.fit_transform(dataset), columns=dataset.columns)
dataset = df_scaled
dataset.tail()


### Frequency Distribution

Create the dimension that we are interested in investigating.

In [ ]:
mtr = ['mtr_1', 'mtr_2', 'mtr_3'] # CS Mentors
prcs = ['prcs_1', 'prcs_2', 'prcs_3', 'prcs_4', 'prcs_5'] # Prior CS Exposure
atcs = ['atcs_1', 'atcs_2', 'atcs_3', 'atcs_5', 'atcs_4', 
        'atcs_6', 'atcs_7', 'atcs_8', 'atcs_9']# self reported attitude about CS competency
atct = ['atct_1', 'atct_2', 'atct_3', 'atct_4', 
        'atct_5', 'atct_6', 'atct_7', 'atct_8'] # Self reported attitudes about computational thinking
blg = ['blg_1', 'blg_2', 'blg_3', 'blg_4'] # Sense of belonging in the class room
clet = ['clet_1', 'clet_2'] # Social implications and ethics
atcsgender = ['atcsgender_1', 'atcsgender_2', 'atcsgender_3'] 
atcsjob = ['atcsjob_1', 'atcsjob_2'] 
cltrcmp = ['cltrcmp_1', 'cltrcmp_2'] # Culutral competency
priorcs10 = 'priorcs10' # had taken CS10 prior


## Summarizing Data

For the majority of this plots, we can see that they are not normally distributed. Most of the data is either skewed to the left or skewed to the right. This lets us know that we can not rely of descriptive statistics like the mean to make sense of the data.

In [ ]:
dataset[atcs].plot(kind='kde')

x = [-0.5, 0.0, 0.5, 1.0, 1.5]
labels = ["", "Strongly Disagree", "Neutral", "Strongly Agree" , ""]
pyplt.xticks(x, labels)

pyplt.xlabel('SURVEY RESPONSES')
pyplt.title('DENSITY ESTIMATION OF COMPUTER SCIENCE ABILITY')
pyplt.legend(loc='upper right', shadow=True, fontsize='medium')
pyplt.savefig('report/figures/atcs_kde.pdf', dpi=100)

In [ ]:
dataset[atcs].quantile(0.50)

In [ ]:
dataset[atct].plot(kind='kde')

x = [-0.5, 0.0, 0.5, 1.0, 1.5]
labels = ["", "Strongly Disagree", "Neutral", "Strongly Agree" , ""]
pyplt.xticks(x, labels)

pyplt.xlabel('SURVEY RESPONSES')
pyplt.title('DENSITY ESTIMATION OF COMPUTATIONAL THINKING ABILITY')
pyplt.legend(loc='upper right', shadow=True, fontsize='medium')
pyplt.savefig('report/figures/atct_kde.pdf', dpi=100)

In [ ]:
dataset[atct].quantile(0.50)

In [ ]:
dataset[blg].plot(kind='kde')

x = [-0.5, 0.0, 0.5, 1.0, 1.5]
labels = ["", "Strongly Disagree", "Neutral", "Strongly Agree" , ""]
pyplt.xticks(x, labels)

pyplt.xlabel('SURVEY RESPONSES')
pyplt.title('DENSITY ESTIMATION OF BELONGING')
pyplt.legend(loc='upper right', shadow=True, fontsize='medium')
pyplt.savefig('report/figures/blg_kde.pdf', dpi=100)

In [ ]:
dataset[blg].quantile(.50)

In [ ]:
dataset[atcsgender].describe().ix['50%']

In [ ]:
dataset[atcsgender].plot(kind='kde')

x = [-0.5, 0.0, 0.5, 1.0, 1.5]
labels = ["", "Strongly Disagree", "Neutral", "Strongly Agree" , ""]
pyplt.xticks(x, labels)

pyplt.xlabel('SURVEY RESPONSES')
pyplt.title('DENSITY ESTIMATION OF ATCSGENDER')
pyplt.legend(loc='upper center', shadow=True, fontsize='medium')
pyplt.savefig('report/figures/atcsgender_kde.pdf', dpi=100)

In [ ]:
dataset[atcsgender].quantile(.50)

In [ ]:
tools.dataLookUp('atcsgender_3')

As we can see, these variables have fair variability in their distributions. 

### Extract features and labels 

Extract feature (X) and target (y) columns

In [ ]:
target_col = dataset['gender_Female']  #  column is the target/label 
y = target_col  # corresponding targets/labels

print "\nLabel values:-"
print y.head()  

In [ ]:
X = dataset.drop(['gender_Female', 'gender_Male'], axis=1, inplace=False)

print "\nFeature values:-"
print X.head()  

In [ ]:
## Make sure we drop the target from the table, otherwise we will be including the labeled data

print "Feature column(s):-", list(X)

### Determine if classes are balanced
- As we can see the dataset is unbalanced, we have more males than females.




In [ ]:
y.plot.hist()

pyplt.grid(False)
_= pyplt.xlabel('VALUE OF TARGET LABEL')
_= pyplt.ylabel('COUNT')
_= pyplt.title('HISTOGRAM OF TARGET CLASS')
_= pyplt.legend(loc='upper center', shadow=True, fontsize='medium')
_= pyplt.yticks(np.arange(0, 700, 100))
pyplt.savefig('report/figures/targetClass.pdf', dpi=100)


In [ ]:
num_male = y.tolist().count(0)
num_female = y.tolist().count(1)


print "number of males in data", num_male
print "number of females in data", num_female


print "ration of males to females {}".format(num_male/ num_female)


In [ ]:
# First, decide how many training vs test samples you want
num_all = X.shape[0]  # same as len(student_data)
num_train = 662  # about 75% of the data
num_test = num_all - num_train

In [ ]:
num_train

In [ ]:
from sklearn import cross_validation



def shuffle_split_data(X, y):
    """ Shuffles and splits data into 75% training and 25% testing subsets,
        then returns the training and testing subsets. """
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, 
                                                                        train_size=num_train, random_state=42)
    
    
    


    # Return the training and testing data subsets
    return X_train, y_train, X_test, y_test

In [ ]:
try:
    X_train, y_train, X_test, y_test = shuffle_split_data(X, y)
    print "Successfully shuffled and split the data!"
except:
    print "Something went wrong with shuffling and splitting the data."


print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])


## Modeling and Validating




I have created four separate tranches to investigate the data:
- All features
- A reduced dataset of features

## Train on all features

In [ ]:
X = X_train

seed = 342 # For reproducability
folds = 50

In [ ]:

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.learning_curve import validation_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics



models = {
          'XGBoost': XGBClassifier(),
          'DecisionTree': DecisionTreeClassifier(),
          'SVC': svm.SVC(),
          'RandomForest': RandomForestClassifier()
         }


In [ ]:
for model_name, model in models.iteritems():
    print model_name
    kfold = StratifiedKFold(y_train, n_folds=folds, random_state=np.random.seed(seed))
    results = cross_val_score(model, X, y_train, cv=kfold, scoring='f1')
    print("Training data accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)), '\n'
    
   

### Predict 

In [ ]:
Features_test = X_test

for model_name, model in models.iteritems():
    print model_name
    # make predictions for test data
    model.fit(X, y_train)
    y_pred = model.predict(Features_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    
    print "F1 score: %.3f%%"% (metrics.f1_score(y_test, predictions))
    
    C = confusion_matrix(y_test, predictions)
    tools.show_confusion_matrix(C, 'report/figures/'+model_name+'.pdf', ['Class Male', 'Class Female'])

## Plot Decision Tree

In [ ]:
import subprocess

def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("report/figures/dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f, feature_names=feature_names, filled=True, rounded=True,  
                         special_characters=True)
    command = ["dot", "-Tpng", "report/figures/dt.dot", "-o", "report/figures/dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [ ]:
dt = DecisionTreeClassifier(random_state=np.random.seed(seed))
dt.fit(X, y_train)

visualize_tree(dt, X.columns)

### Select XGBoost classifier
- Plot first two xgboost trees to see which features were doing the most work of splitting the data
- Generate Features from XGBoost classifier 

In [ ]:
model = XGBClassifier()
model.fit(X, y_train)
g = xgb.to_graphviz(model, num_trees=2, rankdir='LR')
g.render('report/figures/X_graph', view=True)

We are just displaying the first two trees here.   

On simple models the first two trees may be enough. XGBoost generate `k` trees at each round for a `k`-classification problem.

# Tune Estimator

### Ideal choices of parameters as taken from Xgboost With Python ebook

- Number of Trees (`n_estimators`) set to a fixed value between 100 and 1000, depending on the dataset size.
- Learning Rate (`learnin_rate`) simplified to the ratio: [2 to 10]/trees, depending on the trees number of trees.
- Row Sampling (`subsample`) grid searched values in the range [0.5, 0.75, 1.0].
- Column Sampling (`colsample` bytree and maybe colsample bylevel) grid searched values in the range [0.4, 0.6, 0.8, 1.0].
- Min Leaf Weight (`min_child_weight`) simplified to the ratio 3/rare_events , where rare events rare events is the percentage of rare event observations in the dataset.
- Tree Size (`max_depth`) grid searched values in the rage [4, 6, 8, 10].
- Min Split Gain (`gamma`) fixed with a value of zero.


In [ ]:
X = X_train

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import f1_score, make_scorer
from xgboost.sklearn import XGBClassifier



default_params = {
    'objective': 'binary:logistic',
    'max_depth': 1,
    'learning_rate': 0.3
}


n_estimators_range = np.linspace(1, 200, 10).astype('int') 

# Build a stratified shuffle object because of unbalanced data
folds = 50
ssscv = StratifiedShuffleSplit(y_train, folds, random_state=np.random.seed(seed))


train_scores, test_scores = validation_curve(
    XGBClassifier(**default_params),
    X, y_train,
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=ssscv,
    scoring='accuracy'
)


 Show the validation curve plot

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with XGBoost (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.4, 1.1)

plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

i = np.argmax(test_scores_mean)
print("Best cross-validation result ({0:.2f}) obtained for {1} trees".format(test_scores_mean[i], 
                                                                             n_estimators_range[i]))

fig.savefig('report/figures/varianceCurve_1.pdf', format='pdf')


Looking at the plot we can draw the following conclusions:
- training score keeps growing while adding new trees, but from a certain point CV score is fixed
- variance is lowest, and bias is high for less than 25 trees,
- we can see that the model is quite stable keeping variance fixed when increasing it's complexity   

We can assume that the trade-off for our model will be met at n_estimators = 50. The variance is still to big.


Tweak parameters
- To reduce a variance:  
    - each tree we will use 70% randomly chosen samples
    - 60% randomly chosen features   
    
- To decrease the bias (bigger accuracy):
    - Add an extra level to each tree


In [ ]:
default_params = {
    'objective': 'binary:logistic',
    'max_depth': 2, # changed
    'learning_rate': 0.3,
    'colsample_bytree': 0.6, # added
    'subsample': 0.7 # added
}

n_estimators_range = np.linspace(1, 200, 10).astype('int')

train_scores, test_scores = validation_curve(
    XGBClassifier(**default_params),
    X, y_train,
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=ssscv,
    scoring='accuracy'
)

In [ ]:

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with XGBoost (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.4, 1.1)
plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

i = np.argmax(test_scores_mean)
print("Best cross-validation result ({0:.2f}) obtained for {1} trees".format(test_scores_mean[i], 
                                                                             n_estimators_range[i]))
fig.savefig('report/figures/varianceCurve_2.pdf', format='pdf')

In [ ]:
params_grid = {
            'learning_rate': [0.2222, 0.4444, 0.6666, 0.8888],
            'max_depth': [4, 6, 8, 10],
            'n_estimators': range(100, 1100, 100),
            'colsample_bytree': [0.6],
            'subsample':[0.7]
}

params_fixed = {
    'objective': 'binary:logistic',
    'silent': 1
}

In [ ]:
# Load and use already tuned classifier, else tune classifier

tune_flag = False

if tune_flag:
    grid = GridSearchCV(estimator=XGBClassifier(**params_fixed),
        param_grid=params_grid,
        cv=ssscv,
        scoring='f1')
    grid.fit(X, y_train)
    
    print "Best accuracy obtained: {0}".format(grid.best_score_)
    print "Parameters:"
    for key, value in grid.best_params_.items():
        print "\t{}: {}".format(key, value)
    model = grid.best_estimator_
    
    # save model to file
    pickle.dump(model, open("genderedCSExperience.pickle.dat", "wb"))
    
else:    
    model = tools.load_model()
    print model

In [ ]:
Features_test = X_test

# make predictions for test data
model.fit(X, y_train)
y_pred = model.predict(Features_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
print "F1 score: %.2f%%"% (f1_score(y_test, predictions))

In [ ]:
model.fit(X, y_train)
g = xgb.to_graphviz(model, num_trees=2, rankdir='LR')
g.render('report/figures/Tuned_model_graph', view=True)

In [ ]:
C = confusion_matrix(y_test, predictions)
tools.show_confusion_matrix(C, 'report/figures/tuned_model_CM.pdf', ['Class Male', 'Class Female'])